# Assignment \#03

## Loops, Conditionals, and Family

**Outcomes**
1. Review construction of for loops
1. Review conditional statements
1. Explore an HTML tree

**Required Sections**




In [ ]:
import numpy as np

***

We will begin with a further review of for loops. For loops have the basic structure 

for iterator in iterable:
    BLOCK1
BLOCK2

Iterator takes on the different levels of the iterable. The value of iterator will change with each loop. This allows us to execute similar code (BLOCK1) multiple times over different values. 

When we have code after the indentation (BLOCK2) this will return to executing after the last loop. 

Say we want to find the square (power 2) of each number in a list. The below example will exectue this. 

In [ ]:
for x in [1,2,3]:
    print(x**2)

***
### Q1. Write a for loop that will print the cube (power 3) of each value in the list [1,2,3,4].

***

We may also use loops to update or keep track of certain values. For instance, the Fibonacci begins with $1, 1$ and follows where the $n^th$ number is the sequence is the sum of the previous numbers $n-1$ and $n-2$. 

So, the series begins $1,1,2,3,5,8,13,...$. 

We can use variables from outside the loop and update them during the loop in order to find new values. 

Here we initialize three values. We can use them to find the series. `n` is the current value of the series. `nm1` is the previous term, `nm2` is two terms prior. 
So, each loop, we print the iterator $i$ to see how the values change. We find the term `n` by summing the previous terms `nm1` and `nm2`. Then we update those. 

The function `np.arange(l)` creates an iterable object of length `l`. 

Two code chunks are provided. Try adding print statements to the second to see how the values change each loop. Keep one chunk unchanged in case something goes awry. 


In [ ]:
n = 1
nm1 = 1
nm2 = 0
for i in np.arange(5):
    # print the value of the series
    print("The", i+1, "term is ", n)
    n = nm1 + nm2
    nm2 = nm1
    nm1 = n
    

In [ ]:
n = 1
nm1 = 1
nm2 = 0
for i in np.arange(5):
    # print the value of the series
    print("The", i+1, "term is ", n)
    n = nm1 + nm2
    nm2 = nm1
    nm1 = n
    

***
The golden ratio appears in a number of natural phenomena. This ratio is defined algebraically as any two positive numbers $a>b$ such that $\frac{a+b}{a} = \frac{a}{b}$. 

In order to estimate the ratio, we can apply a recursive algorithm. This means we apply the same operation over and over again. 

Given an initial estimate $a$, we can get a new estimate of the golden ratio as $1 + \frac{1}{a}$. 

### Q2. Write a for loop that updates and prints updates of our initial estimate 5 times. 

***
***

Now we consider a short review of conditional statements. 

Conditional statements control the flow of code. Python focuses on `if`, `elif`, and `else` statements. 

The code following statements only evaluates if the expression in the `if` portion is True. The general form is given below. 

`
if cond1:
    block1
elif cond2:
    block2
elif cond3:
    block3
else:
    block4`

Python will work through these statements in sequence. If the first condition is true, block1 will evaluate and the rest will be skipped. If the first condition is false, the subsequent conditions will be evaluated until a true one is found. Once a true statement is found, the respective block will be executed. 
If none of the conditions are true, then block under the else statement will run. 

### Q3: Change all ellipses in the next code block to either `True` or `False` so that the statement `if 1` prints.  

In [ ]:
if ...:
    print("if 1")
elif ...:
    print("elif 1")
elif ...:
    print("elif 2")
elif ...:
    print("elif 3")
else:
    print("else")

### Q4: Change all ellipses in the next code block to either `True` or `False` so that the statement `elif 1` prints.  

In [ ]:
if ...:
    print("if 1")
elif ...:
    print("elif 1")
elif ...:
    print("elif 2")
elif ...:
    print("elif 3")
else:
    print("else")

### Q5: Change all ellipses in the next code block to either `True` or `False` so that the statement `elif 2` prints.  

In [ ]:
if ...:
    print("if 1")
elif ...:
    print("elif 1")
elif ...:
    print("elif 2")
elif ...:
    print("elif 3")
else:
    print("else")

### Q6: Change all ellipses in the next code block to either `True` or `False` so that the statement `elif 3` prints.  

In [ ]:
if ...:
    print("if 1")
elif ...:
    print("elif 1")
elif ...:
    print("elif 2")
elif ...:
    print("elif 3")
else:
    print("else")

### Q7: Change all ellipses in the next code block to either `True` or `False` so that the statement `else` prints.  

In [ ]:
if ...:
    print("if 1")
elif ...:
    print("elif 1")
elif ...:
    print("elif 2")
elif ...:
    print("elif 3")
else:
    print("else")

***
***
## Explore an HTML tree



In [10]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, NavigableString, Tag

url = "https://pythonscraping.com/pages/page3.html"
html = urlopen(url)
bs = BeautifulSoup(html, 'html.parser')

***

We talked in class about exploring the children of trees. Now we will discuss navigating across siblings and parents. 

### Q8: Begin by creating a new variable `table` that takes the value of the Tag `table` which is the child of div, child of body.  

Hint: You may use the `bs.tag.tag.tag...` syntax. 

In class, we talked about how to loop over the children of tag. Try doing this now. 

### Q9: Using a for loop, print the `name` of all the children of table. 

Hint: they should alternate between two value. 



***

We may want to be able to go back up the tree, rather than down. Similar to how `.children` will return all the children of a tag, `.parent` will give us the tag that occurs above. 

### Q10: Try using `.parent` to print the tag that includes the table. 

Finally, We can also move horizontally through the siblings at the same level of the HTML tree. 

Beautiful Soup offers us four methods to do such:

| Syntax      | Description |
| ----------- | ----------- |
| .next_sibling      | Returns immediately following sibling       |
| .previous_sibling   | Returns immediately preceeding sibling        |
| .next_siblings      | Returns an iterable object of all following siblings       |
| .previous_siblings   | Returns an iterable object of all preceeding siblings        |

Try using these methods on `table.tr`. We can also stack these

### Q11: Try all four methods on `table.tr` in the box below. What differs from their output. 

**Answer:**

 We can also stack these calls (e.g. `table.next_sibling.next_sibling`) in a single line of code to jump multiple siblings. 
 
 ### Q11: How many times must you add `.next_sibling` to `table.tr` to get the gift `Russian Nesting Dolls`? Put the code and your written answer in the subsequent boxes. 

**Answer:**